In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from fractions import Fraction

from src.interface_representation.interface_types import InterfaceType


def create_dice_error_box_plots_mu(df, ylims=[(0.5, 1), (0.95, 1), (0.95, 1)]):
    """
    Create box plots comparing different interface types across mu values.

    Parameters:
    -----------
    df : pandas.DataFrame
        DataFrame containing columns: 'mu', 'interface_type', 'epsilon', 'Dice'
    ylims : list of tuples
        List of (ymin, ymax) tuples for each subplot

    Returns:
    --------
    fig : matplotlib.figure.Figure
        The created figure object
    """

    def float_to_fraction(float_num):
        fraction = Fraction(float_num).limit_denominator()
        return f"{fraction.numerator}/{fraction.denominator}"

    # Get unique mu values
    mus = sorted(df['mu'].unique())

    # Create dictionary to store data grouped by mu
    mu_to_data = {mu: {} for mu in mus}

    # Process each row in the DataFrame
    for _, row in df.iterrows():
        mu = row['mu']
        interface_type = row['interface_type']
        epsilon = row['epsilon']
        dice_score = row['dice']

        # Create label based on interface type
        if interface_type == InterfaceType.TANH_EPSILON.name:
            label = f'Tanh {epsilon}'
        elif interface_type == InterfaceType.SIGNED_DISTANCE_EXACT.name:
            label = 'SDF'
        elif interface_type == InterfaceType.SIGNED_DISTANCE_APPROXIMATE.name:
            label = 'SDF approx'
        elif interface_type == InterfaceType.HEAVISIDE.name:
            label = 'Sharp'
        else:
            raise ValueError(f"Unknown interface type: {interface_type}")

        if label not in mu_to_data[mu]:
            mu_to_data[mu][label] = []
        mu_to_data[mu][label].append(dice_score)

    # Sort and process labels
    for mu in mu_to_data:
        # Sort by interface type
        mu_to_data[mu] = dict(sorted(mu_to_data[mu].items()))

        # Convert epsilon values to fractions in labels
        new_data = {}
        for k, v in mu_to_data[mu].items():
            if k.startswith('Tanh'):
                epsilon = float(k.split(' ')[1])
                new_data[f'Tanh {float_to_fraction(epsilon)}'] = v
            else:
                new_data[k] = v
        mu_to_data[mu] = new_data

    # Create plots
    fig, axs = plt.subplots(1, len(mus), figsize=(12, 3), dpi=200)

    for i, mu in enumerate(mus):
        data = [mu_to_data[mu][label] for label in mu_to_data[mu]]
        labels = list(mu_to_data[mu].keys())

        axs[i].boxplot(data, tick_labels=labels)
        axs[i].set_title(f'$\\mu$={mu} dataset')
        axs[i].set_xlabel('Interface Type')
        axs[i].set_ylabel('Dice Coefficient')
        axs[i].spines['top'].set_visible(False)
        axs[i].spines['right'].set_visible(False)
        axs[i].tick_params(axis='x', rotation=90)

        if i < len(ylims):
            axs[i].set_ylim(*ylims[i])

    plt.tight_layout()
    plt.savefig('figures/fig_dice_error_box_plots_mu.png')
    plt.show()


df = pd.read_csv('evaluation_results_2.csv')
create_dice_error_box_plots_mu(df)

In [ ]:
def create_hausdorff_error_box_plots_mu(df, ylims=[(0, 35), (0, 7), (0, 4)]):
    """
    Create box plots comparing different interface types across mu values using Hausdorff distance.
    Units of voxels.

    Parameters:
    -----------
    df : pandas.DataFrame
        DataFrame containing columns: 'mu', 'interface_type', 'epsilon', 'hausdorff'
    ylims : list of tuples
        List of (ymin, ymax) tuples for each subplot

    Returns:
    --------
    fig : matplotlib.figure.Figure
        The created figure object
    """

    def float_to_fraction(float_num):
        fraction = Fraction(float_num).limit_denominator()
        return f"{fraction.numerator}/{fraction.denominator}"

    # Get unique mu values
    mus = sorted(df['mu'].unique())

    # Create dictionary to store data grouped by mu
    mu_to_data = {mu: {} for mu in mus}

    # Process each row in the DataFrame
    for _, row in df.iterrows():
        mu = row['mu']
        interface_type = row['interface_type']
        epsilon = row['epsilon']
        hausdorff_dist = row['hausdorff'] * 64  # TODO: remove magic number

        # Create label based on interface type
        if interface_type == InterfaceType.TANH_EPSILON.name:
            label = f'Tanh {epsilon}'
        elif interface_type == InterfaceType.SIGNED_DISTANCE_EXACT.name:
            label = 'SDF'
        elif interface_type == InterfaceType.SIGNED_DISTANCE_APPROXIMATE.name:
            label = 'SDF approx'
        elif interface_type == InterfaceType.HEAVISIDE.name:
            label = 'Sharp'
        else:
            raise ValueError(f"Unknown interface type: {interface_type}")

        if label not in mu_to_data[mu]:
            mu_to_data[mu][label] = []
        mu_to_data[mu][label].append(hausdorff_dist)

    # Sort and process labels
    for mu in mu_to_data:
        # Sort by interface type
        mu_to_data[mu] = dict(sorted(mu_to_data[mu].items()))

        # Convert epsilon values to fractions in labels
        new_data = {}
        for k, v in mu_to_data[mu].items():
            if k.startswith('Tanh'):
                epsilon = float(k.split(' ')[1])
                new_data[f'Tanh {float_to_fraction(epsilon)}'] = v
            else:
                new_data[k] = v
        mu_to_data[mu] = new_data

    # Create plots
    fig, axs = plt.subplots(1, len(mus), figsize=(12, 3), dpi=200)

    for i, mu in enumerate(mus):
        data = [mu_to_data[mu][label] for label in mu_to_data[mu]]
        labels = list(mu_to_data[mu].keys())

        axs[i].boxplot(data, tick_labels=labels)
        axs[i].set_title(f'$\\mu$={mu} dataset')
        axs[i].set_xlabel('Interface Type')
        axs[i].set_ylabel('Hausdorff Distance')
        axs[i].spines['top'].set_visible(False)
        axs[i].spines['right'].set_visible(False)
        axs[i].tick_params(axis='x', rotation=90)

        if i < len(ylims):
            axs[i].set_ylim(*ylims[i])

    plt.tight_layout()
    plt.savefig('figures/fig_hausdorff_error_box_plots_mu.png')
    plt.show()


df = pd.read_csv('evaluation_results_2.csv')
create_hausdorff_error_box_plots_mu(df)